In [65]:
import csv
import pandas as pd

In [2]:
extract = pd.read_csv("egcut_epi_nesma_extracted_features_example.csv",header=None)

In [3]:
extract.columns = ["text_ID","start","end","object","value","unit","min","max"]

In [4]:
text = pd.read_csv("egcut_epi_nesma_texts_example.csv",header=None)

In [5]:
text.columns = ["text_ID","text"]
text.head()

,text_ID,text
0,347474,\nRR 132/84 \n\n
1,46809,RR 130/80 mmHg.
2,45389,"Kergelt liigkaaluline. Nahk, limaskestad taval..."
3,47482,"RR142/89mmHg, HR 76 x min, p167cm, k89kg. EKG..."
4,53393,"RR143/87mmHg, HR60 x min, p180cm, k93kg"


In [ ]:
text.text_ID.value_counts()

In [6]:
text = text.drop_duplicates(subset='text_ID', keep='first')

In [8]:
DS = pd.merge(extract, text, how="left", on='text_ID')

In [9]:
X = DS.text
Y = DS.object
Z = DS.value
X = X[:5000]
Y = Y[:5000]
Z = Z[:5000]

In [10]:
import numpy as np
from keras.utils import np_utils

import matplotlib.pyplot as plt
from keras.layers import GRU,Dropout,BatchNormalization
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

%matplotlib inline

Using TensorFlow backend.


In [11]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [12]:
Xtrain, Xtest, Ytrain, Ytest , Ztrain, Ztest = train_test_split(X, Y, Z, test_size=0.2, random_state=1)

Xtrain, Xval, Ytrain, Yval , Ztrain, Zval= train_test_split(Xtrain, Ytrain, Ztrain, test_size=0.25, random_state=1)



In [13]:
shape = 3
encoder = LabelEncoder()
encoder.fit(Y) #3 classes in Object column


Ytrain = encoder.transform(Ytrain)
Ytest = encoder.transform(Ytest)
Yval = encoder.transform(Yval)

Ytrain = np_utils.to_categorical(Ytrain)
Ytest = np_utils.to_categorical(Ytest)
Yval = np_utils.to_categorical(Yval)



In [14]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(Xtrain)

Xtrain = tokenizer.texts_to_sequences(Xtrain)
Xtest = tokenizer.texts_to_sequences(Xtest)
Xval = tokenizer.texts_to_sequences(Xval)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index



In [16]:
maxlen = 100

Xtrain = pad_sequences(Xtrain, padding='post', maxlen=maxlen)
Xtest = pad_sequences(Xtest, padding='post', maxlen=maxlen)
Xval = pad_sequences(Xval, padding='post', maxlen=maxlen)



In [68]:
from keras.models import Sequential
from keras import layers
from keras.layers import Input, LSTM, Embedding, Dense

model = Sequential()
# model.add(keras.layers.LSTM(100, input_shape=(Xtrain.shape[0], Xtrain.shape[1]), return_sequences='true'))
# # model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.LSTM(250,return_sequences=True))
# # model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.Dense(64))
# model.add(keras.layers.Activation("linear"))
# model.add(keras.layers.Dense(2, activation='linear'))
# model.add(keras.layers.Dense(1, activation='linear'))

input_text = keras.layers.Input(shape=(100,))
emb = Embedding(input_dim=37962, output_dim=256, input_length=100)(input_text)

x = keras.layers.LSTM(256)(emb)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dense(32, activation='relu')(x)
# x = keras.layers.Flatten()(x)
output_object = keras.layers.Dense(3, activation='linear')(x)
output_value = keras.layers.Dense(1, activation='linear')(x)


model = keras.models.Model(inputs=input_text,
                                outputs=[output_object,output_value])

model.compile(optimizer='adam',loss=[ "categorical_crossentropy","mean_squared_error"],metrics=['accuracy'])
model.summary()


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 100, 256)     9718272     input_12[0][0]                   
__________________________________________________________________________________________________
lstm_8 (LSTM)                   (None, 256)          525312      embedding_8[0][0]                
__________________________________________________________________________________________________
dense_24 (Dense)                (None, 64)           16448       lstm_8[0][0]                     
____________________________________________________________________________________________

In [25]:
tryx = Xtrain.reshape((3000,100))
history = model.fit(tryx, y = [Ytrain,Ztrain],
                    epochs=10,
                    verbose=True,
                    validation_data=(Xval, [Yval,  Zval]),batch_size=1000)
#,batch_size=5000


Train on 3000 samples, validate on 1000 samples
Epoch 1/10
3000/3000 [==============================] - 19s 6ms/step - loss: 10457817.5339 - dense_22_loss: 1.0908 - dense_23_loss: 10457816.0000 - dense_22_accuracy: 0.5630 - dense_23_accuracy: 0.0010 - val_loss: 207296.1875 - val_dense_22_loss: 1.0937 - val_dense_23_loss: 207295.0938 - val_dense_22_accuracy: 0.5520 - val_dense_23_accuracy: 0.0000e+00
Epoch 2/10
3000/3000 [==============================] - 19s 6ms/step - loss: 10456130.1055 - dense_22_loss: 1.0982 - dense_23_loss: 10456129.0000 - dense_22_accuracy: 0.5630 - dense_23_accuracy: 0.0020 - val_loss: 206015.5156 - val_dense_22_loss: 1.1051 - val_dense_23_loss: 206014.4062 - val_dense_22_accuracy: 0.5520 - val_dense_23_accuracy: 0.0000e+00
Epoch 3/10
3000/3000 [==============================] - 20s 7ms/step - loss: 10453387.5869 - dense_22_loss: 1.1066 - dense_23_loss: 10453386.0000 - dense_22_accuracy: 0.5630 - dense_23_accuracy: 0.0027 - val_loss: 204569.8594 - val_dense_22_l

In [27]:
# f= open("report.csv","a")
# loss1, accuracy1,loss2, accuracy2 = 
model.evaluate(Xtrain, [Ytrain,Ztrain], verbose=True)
# print("Training Accuracy: {:.4f}".format(accuracy))
# f.write(",{:.4f}".format(accuracy))

# loss, accuracy = model.evaluate(Xval, Yval, verbose=True)

# print("Validation Accuracy:  {:.4f}".format(accuracy))
# f.write(",{:.4f} \n".format(accuracy))
# f.close()
# plot_history(history)
# Ypred = model.predict(Xtest)
# from sklearn.metrics import confusion_matrix, classification_report

# matrix = confusion_matrix(Ytest.argmax(axis=1), Ypred.argmax(axis=1))
# classification_Report = classification_report(Ytest.argmax(axis=1), Ypred.argmax(axis=1), output_dict=True)
# df = pd.DataFrame(classification_Report).transpose()
# df.to_csv("ClassificationReport.csv")

3000/3000 [==============================] - 17s 6ms/step


[10428394.656989584,
 1.0307753086090088,
 10400667.0,
 0.5630000233650208,
 0.0026666666381061077]

In [47]:
pred1.shape

(1000, 3)

In [42]:
pred1,pred2 = model.predict(Xval)


In [50]:
Xtrain.shape

(3000, 100)

In [57]:
Ytrain.shape

(3000, 3)

In [51]:
model.evaluate(Xtest, [Ytest,Ztest], verbose=True)


1000/1000 [==============================] - 6s 6ms/step


[15979794.698929688,
 1.0298240184783936,
 15605314.0,
 0.5659999847412109,
 0.004000000189989805]

In [62]:
pred.reshape

(1000, 1)

In [ ]:
# np.argmax(pred1, axis=-1)
